# A notebook for test and debug some python code

### To do
Create a conda env 
    Im having a error
Selecionar e colocar arquiteturas
Colocar funcoes de perda da tabela
Colocar todas as metricas

### import the utils libs

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torchviz as tv

## Archtectures


### U-net

In [8]:


class DoubleConv(nn.Module):
    """
    Bloco de duas convoluções 3x3 seguidas de ReLU.
    """
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    """
    Implementação da arquitetura U-Net adaptada para geração de imagens.
    """
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        # Encoder
        self.down1 = DoubleConv(in_channels, 64)
        self.down2 = DoubleConv(64, 128)
        self.down3 = DoubleConv(128, 256)
        self.down4 = DoubleConv(256, 512)
        self.down5 = DoubleConv(512, 1024)

        # Max-pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Decoder
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv_up1 = DoubleConv(1024, 512)
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv_up2 = DoubleConv(512, 256)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv_up3 = DoubleConv(256, 128)
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv_up4 = DoubleConv(128, 64)

        # Final convolution
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        d1 = self.down1(x)
        d2 = self.down2(self.pool(d1))
        d3 = self.down3(self.pool(d2))
        d4 = self.down4(self.pool(d3))
        d5 = self.down5(self.pool(d4))

        # Decoder
        u1 = self.up1(d5)
        u1 = torch.cat((u1, d4), dim=1)
        u1 = self.conv_up1(u1)
        u2 = self.up2(u1)
        u2 = torch.cat((u2, d3), dim=1)
        u2 = self.conv_up2(u2)
        u3 = self.up3(u2)
        u3 = torch.cat((u3, d2), dim=1)
        u3 = self.conv_up3(u3)
        u4 = self.up4(u3)
        u4 = torch.cat((u4, d1), dim=1)
        u4 = self.conv_up4(u4)

        return self.final_conv(u4)

if __name__ == "__main__":
    # Definindo o dispositivo (GPU se disponível)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Inicializando a U-Net
    in_channels = 3  # Por exemplo, RGB
    out_channels = 3  # Saída de imagem RGB
    model = UNet(in_channels, out_channels).to(device)

    # Exemplo de entrada
    x = torch.randn(1, in_channels, 256, 256).to(device)  # Batch size 1, 256x256 imagem
    output = model(x)

    print(x.shape, output.shape)  # Deve ser (1, out_channels, 256, 256)
     # Plotando a arquitetura da rede
    


torch.Size([1, 3, 256, 256]) torch.Size([1, 3, 256, 256])


In [10]:
try:
        y = model(torch.randn(1, in_channels, 256, 256).to(device))
        tv.make_dot(y, params=dict(list(model.named_parameters()))).render("unet_architecture", format="png")
        
        #plot.format = 'png'
        
        print("Arquitetura da U-Net plotada com sucesso.")
except ImportError:
        print("Para plotar a arquitetura, instale o pacote torchviz.")


Arquitetura da U-Net plotada com sucesso.


In [51]:
summary(model, x,depth=1, dtypes=[torch.long],branching=False,verbose=1, col_width=16)

Layer (type:depth-idx)                   Output Shape     Param #
DoubleConv: 2-1                          [-1, 64, 256, 256] --
MaxPool2d: 1-1                           [-1, 64, 128, 128] --
DoubleConv: 2-2                          [-1, 128, 128, 128] --
MaxPool2d: 1-2                           [-1, 128, 64, 64] --
DoubleConv: 2-3                          [-1, 256, 64, 64] --
MaxPool2d: 1-3                           [-1, 256, 32, 32] --
DoubleConv: 1-4                          [-1, 512, 32, 32] --
Sequential: 2-4                          [-1, 512, 32, 32] --
ConvTranspose2d: 2-5                     [-1, 256, 64, 64] 524,544
DoubleConv: 2-6                          [-1, 256, 64, 64] --
ConvTranspose2d: 2-7                     [-1, 128, 128, 128] 131,200
DoubleConv: 2-8                          [-1, 128, 128, 128] --
ConvTranspose2d: 2-9                     [-1, 64, 256, 256] 32,832
DoubleConv: 2-10                         [-1, 64, 256, 256] --
Conv2d: 1-5                              [

Layer (type:depth-idx)                   Output Shape     Param #
DoubleConv: 2-1                          [-1, 64, 256, 256] --
MaxPool2d: 1-1                           [-1, 64, 128, 128] --
DoubleConv: 2-2                          [-1, 128, 128, 128] --
MaxPool2d: 1-2                           [-1, 128, 64, 64] --
DoubleConv: 2-3                          [-1, 256, 64, 64] --
MaxPool2d: 1-3                           [-1, 256, 32, 32] --
DoubleConv: 1-4                          [-1, 512, 32, 32] --
Sequential: 2-4                          [-1, 512, 32, 32] --
ConvTranspose2d: 2-5                     [-1, 256, 64, 64] 524,544
DoubleConv: 2-6                          [-1, 256, 64, 64] --
ConvTranspose2d: 2-7                     [-1, 128, 128, 128] 131,200
DoubleConv: 2-8                          [-1, 128, 128, 128] --
ConvTranspose2d: 2-9                     [-1, 64, 256, 256] 32,832
DoubleConv: 2-10                         [-1, 64, 256, 256] --
Conv2d: 1-5                              [

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import matplotlib.pyplot as plt

class DoubleConv(nn.Module):
    """
    Bloco de duas convoluções 3x3 seguidas de ReLU.
    Permite a parametrização do uso de BatchNorm.
    """
    def __init__(self, in_channels, out_channels, use_batch_norm=True):
        super(DoubleConv, self).__init__()
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        ]
        if use_batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))

        layers.extend([
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        ])
        if use_batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))
        
        self.double_conv = nn.Sequential(*layers)

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    """
    Implementação da arquitetura U-Net parametrizável para geração de imagens.
    Permite a parametrização do número de blocos de convolução e o uso de BatchNorm.
    """
    def __init__(self, in_channels, out_channels, base_filters=64, num_layers=5, use_batch_norm=True):
        super(UNet, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.base_filters = base_filters
        self.num_layers = num_layers
        self.use_batch_norm = use_batch_norm

        # Encoder
        self.encoder_layers = nn.ModuleList()
        filters = base_filters
        
        for i in range(num_layers-1):
            self.encoder_layers.append(DoubleConv(in_channels, filters, use_batch_norm))
            in_channels = filters
            filters *= 2
        
        # Max-pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bottleneck
        self.bottleneck = DoubleConv(filters // 2, filters, use_batch_norm)

        # Decoder
        self.up_layers = nn.ModuleList()
        self.up_convs = nn.ModuleList()
        
        
        for i in range(num_layers-1, 0, -1):#range(num_layers-1, 0, -1)
            filters //= 2
            self.up_layers.append(nn.ConvTranspose2d(filters * 2, filters, kernel_size=2, stride=2))
            self.up_convs.append(DoubleConv(filters * 2, filters , use_batch_norm))
            

        # Final convolution
        
        self.final_conv = nn.Conv2d(base_filters, out_channels, kernel_size=1)
    def see_model(self):
        print(f"""
        encoder: {self.encoder_layers}
        pool: {self.pool}
        botleneck: {self.bottleneck}
        up laeyers: {self.up_layers}
        up_convs: {self.up_convs}
        final conv: {self.final_conv}""")

    def forward(self, x):
        # Encoder
        encoder_results = []
        for layer in self.encoder_layers:
            x = layer(x)
            encoder_results.append(x)
            x = self.pool(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Decoder
        for i, (up, conv) in enumerate(zip(self.up_layers, self.up_convs)):
            x = up(x)
            x = torch.cat((x, encoder_results[-(i+1)]), dim=1)
            x = conv(x)

        return self.final_conv(x)

def standardize_image(image):
    """
    Função para padronizar as imagens de entrada.
    """
    mean = image.mean()
    std = image.std()
    return (image - mean) / std

if __name__ == "__main__":
    # Definindo o dispositivo (GPU se disponível)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Inicializando a U-Net parametrizável
    in_channels = 3  # Por exemplo, RGB
    out_channels = 3  # Saída de imagem RGB
    base_filters = 64
    num_layers = 4
    use_batch_norm = True
    model = UNet(in_channels, out_channels, base_filters, num_layers, use_batch_norm).to(device)
    #model.see_model()

    # Exibindo a arquitetura da rede
    #summary(model, input_size=(in_channels, 256, 256))

    # Exemplo de entrada
    x = torch.randn(1, in_channels, 256, 256).to(device)  # Batch size 1, 256x256 imagem
    # x = standardize_image(x)
    output = model(x)

    print(output.shape)  # Deve ser (1, out_channels, 256, 256)

    # Plotando a arquitetura da rede
    try:

        y = model(torch.randn(1, in_channels, 256, 256).to(device))
        tv.make_dot(y, params=dict(list(model.named_parameters()))).render("unet_architecture_2", format="png")
        print("Arquitetura da U-Net plotada com sucesso.")
    except ImportError:
        print("Para plotar a arquitetura, instale o pacote torchviz.")


torch.Size([1, 3, 256, 256])
Arquitetura da U-Net plotada com sucesso.


In [48]:
summary?

Signature:
summary(
    model: torch.nn.modules.module.Module,
    input_data: Union[torch.Tensor, torch.Size, Sequence[torch.Tensor], Sequence[Union[int, Sequence[Any], torch.Size]], NoneType] = None,
    *args: Any,
    batch_dim: Optional[int] = 0,
    branching: bool = True,
    col_names: Optional[Iterable[str]] = None,
    col_width: int = 25,
    depth: int = 3,
    device: Optional[torch.device] = None,
    dtypes: Optional[List[torch.dtype]] = None,
    verbose: int = 1,
    **kwargs: Any,
) -> torchsummary.model_statistics.ModelStatistics
Docstring:
Summarize the given PyTorch model. Summarized information includes:
    1) Layer names,
    2) input/output shapes,
    3) kernel shape,
    4) # of parameters,
    5) # of operations (Mult-Adds)

Args:
    model (nn.Module):
            PyTorch model to summarize. The model should be fully in either train()
            or eval() mode. If layers are not all in the same mode, running summary
            may have side effects on bat

In [50]:
summary(model, x,depth=1, dtypes=[torch.long],branching=False,verbose=1, col_width=16)

Layer (type:depth-idx)                   Output Shape     Param #
DoubleConv: 2-1                          [-1, 64, 256, 256] --
MaxPool2d: 1-1                           [-1, 64, 128, 128] --
DoubleConv: 2-2                          [-1, 128, 128, 128] --
MaxPool2d: 1-2                           [-1, 128, 64, 64] --
DoubleConv: 2-3                          [-1, 256, 64, 64] --
MaxPool2d: 1-3                           [-1, 256, 32, 32] --
DoubleConv: 1-4                          [-1, 512, 32, 32] --
Sequential: 2-4                          [-1, 512, 32, 32] --
ConvTranspose2d: 2-5                     [-1, 256, 64, 64] 524,544
DoubleConv: 2-6                          [-1, 256, 64, 64] --
ConvTranspose2d: 2-7                     [-1, 128, 128, 128] 131,200
DoubleConv: 2-8                          [-1, 128, 128, 128] --
ConvTranspose2d: 2-9                     [-1, 64, 256, 256] 32,832
DoubleConv: 2-10                         [-1, 64, 256, 256] --
Conv2d: 1-5                              [

Layer (type:depth-idx)                   Output Shape     Param #
DoubleConv: 2-1                          [-1, 64, 256, 256] --
MaxPool2d: 1-1                           [-1, 64, 128, 128] --
DoubleConv: 2-2                          [-1, 128, 128, 128] --
MaxPool2d: 1-2                           [-1, 128, 64, 64] --
DoubleConv: 2-3                          [-1, 256, 64, 64] --
MaxPool2d: 1-3                           [-1, 256, 32, 32] --
DoubleConv: 1-4                          [-1, 512, 32, 32] --
Sequential: 2-4                          [-1, 512, 32, 32] --
ConvTranspose2d: 2-5                     [-1, 256, 64, 64] 524,544
DoubleConv: 2-6                          [-1, 256, 64, 64] --
ConvTranspose2d: 2-7                     [-1, 128, 128, 128] 131,200
DoubleConv: 2-8                          [-1, 128, 128, 128] --
ConvTranspose2d: 2-9                     [-1, 64, 256, 256] 32,832
DoubleConv: 2-10                         [-1, 64, 256, 256] --
Conv2d: 1-5                              [

### Limpa memoria CUDA

In [52]:
import gc

# Verifique a memória alocada e reservada
print(f'Memória alocada: {torch.cuda.memory_allocated()} bytes')
print(f'Memória reservada: {torch.cuda.memory_reserved()} bytes')

# Force a coleta de lixo para liberar a memória imediatamente
gc.collect()

# Libere o cache de memória do PyTorch
torch.cuda.empty_cache()

# Verifique a memória alocada e reservada
print(f'Memória alocada: {torch.cuda.memory_allocated()} bytes')
print(f'Memória reservada: {torch.cuda.memory_reserved()} bytes')

print(f"Memoria limpa")

Memória alocada: 1007716352 bytes
Memória reservada: 1392508928 bytes
Memória alocada: 1007716352 bytes
Memória reservada: 1270874112 bytes
Memoria limpa


### DDPM

### CycleGAN


### VAE

### Vit

## Losses

### Gan loss	 	 


### color loss	 	 



### SSIM	 	 


### MS-SSIM	 		 


### MAE	 		 	 	 


### MSE	 	 	 	 	 


### Uranker	 	 	 	 	 
### MUSIQ	 	 	 	 	 
### Content Loss VGG	 	 	 	 	 
### Acumlative Superiority	 	 	 	 	 
### Discriminative	 	 	 	 	 
### L2	 	 	 	 	 
### L1	 	 	 	 	 
### CIoU	 	 	 	 	 
### Adversarial Loss	 	 	 	 	 
### WGan Loss	 	 	 	 	 
### Gradient Loss	 	 	 	 	 
### Color Structure Perceptual	 	 	 	 	 
### Detail loss	 	 	 	 	 
### CGan	 	 	 	 	 
### Angular	 	 	 	 	 
### margin-ranking loss	 	 	 	 	 
### Classifier loss	 	 	 	 	 
### Identity mapping	 	 	 	 	 
### Cycle consistency loss	 	 	 	 	 
### LAB loss	 	 	 	 	 
### LCH loss

## Metrics